# Event Hooks

https://2.python-requests.org/en/master/user/advanced/#event-hooks

## Only awailable since `requests>=1.0`

`response`:
    The response generated from a Request. 

## Additional hooks for `requests<1.0` (outdated ages ago not recomended to use this versions of requests)

`args`:
    A dictionary of the arguments being sent to Request().

`pre_request`:
    The Request object, directly before being sent.

`post_request`:
    The Request object, directly after being sent.


### `response` hook

In [1]:
import requests

In [2]:
with requests.Session() as s:
    print(s.hooks)


{'response': []}


#### Default sifnature for requests 'responce' hook

```function(r, *args, **kwargs)```

In [3]:
def r_type_hook(r, *args, **kwargs):
    print(f"r_type_hook: {id(r) = }, {type(r) = }, {r = }")
    return r

In [4]:
with requests.Session() as s:
    s.hooks['response'].append(r_type_hook)
    res = s.get("https://httpbin.org/status/500")
    
f"{id(res) = }, {type(res) = }, {res = }"

r_type_hook: id(r) = 140396375174016, type(r) = <class 'requests.models.Response'>, r = <Response [500]>


"id(res) = 140396375174016, type(res) = <class 'requests.models.Response'>, res = <Response [500]>"

[requests.models.Response](https://requests.readthedocs.io/en/master/_modules/requests/models/#Response) - Last source version (your actual version could be lower)

In [5]:
def forget_return_r_hook(r, *args, **kwargs):
    print(f"forget_return_r_hook: {id(r) = }, {type(r) = }, {r = }")


In [6]:
with requests.Session() as s:
    s.hooks['response'].append(forget_return_r_hook)
    res = s.get("https://httpbin.org/status/500")
    
f"{id(res) = }, {type(res) = }, {res = }"

forget_return_r_hook: id(r) = 140396377657248, type(r) = <class 'requests.models.Response'>, r = <Response [500]>


"id(res) = 140396377657248, type(res) = <class 'requests.models.Response'>, res = <Response [500]>"

In [7]:
with requests.Session() as s:
    s.hooks['response'].append(forget_return_r_hook)
    s.hooks['response'].append(r_type_hook)
    res = s.get("https://httpbin.org/status/500")
    
res

forget_return_r_hook: id(r) = 140396375228320, type(r) = <class 'requests.models.Response'>, r = <Response [500]>
r_type_hook: id(r) = 140396375228320, type(r) = <class 'requests.models.Response'>, r = <Response [500]>


<Response [500]>

In [8]:
def raise_on_error(r, *args, **kwargs):
    r.raise_for_status()
    return r


with requests.Session() as s:
    # s.hooks['response'] is list so common operation for list work here
    #  s.append(el) - append to end of hook list
    #  s.hooks['response'] = [] - clean existed hooks
    #  s.hooks['response'] = [new_hook] - overwrite hooks
    #  s.insert(ix, new_hook) - insert new_hook at a given position
    # and etc: https://docs.python.org/3/tutorial/datastructures.html#more-on-lists
    #
    # Note: list are mutable type.
    
    s.hooks['response'].insert(0, raise_on_error)
    
    res = s.get("https://httpbin.org/status/500")
    print("Do after responce")

HTTPError: 500 Server Error: INTERNAL SERVER ERROR for url: https://httpbin.org/status/500

In [9]:
with requests.Session() as s:
    # s.hooks['response'] is list so common operation for list work here
    #  s.append(el) - append to end of hook list
    #  s.hooks['response'] = [] - clean existed hooks
    #  s.hooks['response'] = [new_hook] - overwrite hooks
    #  s.insert(ix, new_hook) - insert new_hook at a given position
    # and etc: https://docs.python.org/3/tutorial/datastructures.html#more-on-lists
    #
    # Note: list are mutable type.
    
    s.hooks['response'] = raise_on_error
    
    res = s.get("https://httpbin.org/status/500")
    print("Do after responce")

HTTPError: 500 Server Error: INTERNAL SERVER ERROR for url: https://httpbin.org/status/500

In [10]:
def print_kwargs(r, *args, **kwargs):
    print(f"{r = }, {kwargs = }")
    return r


with requests.Session() as s:
    s.hooks['response'].insert(0, print_kwargs)
    res = s.get("https://httpbin.org/status/500")

r = <Response [500]>, kwargs = {'timeout': None, 'verify': True, 'proxies': OrderedDict(), 'stream': False, 'cert': None}


### `pre_request` - homebrew hook

```py
import requests


# Prepare data
request = requests.Request('POST', url, ...)
request = hook_function(request)
prepared = request.prepare()

# Send data
session = requests.session()
resp = session.send(prepared)
```

In [11]:
help(requests.models.Request)

Help on class Request in module requests.models:

class Request(RequestHooksMixin)
 |  Request(method=None, url=None, headers=None, files=None, data=None, params=None, auth=None, cookies=None, hooks=None, json=None)
 |  
 |  A user-created :class:`Request <Request>` object.
 |  
 |  Used to prepare a :class:`PreparedRequest <PreparedRequest>`, which is sent to the server.
 |  
 |  :param method: HTTP method to use.
 |  :param url: URL to send.
 |  :param headers: dictionary of headers to send.
 |  :param files: dictionary of {filename: fileobject} files to multipart upload.
 |  :param data: the body to attach to the request. If a dictionary or
 |      list of tuples ``[(key, value)]`` is provided, form-encoding will
 |      take place.
 |  :param json: json for the body to attach to the request (if files or data is not specified).
 |  :param params: URL parameters to append to the URL. If a dictionary or
 |      list of tuples ``[(key, value)]`` is provided, form-encoding will
 |      

[requests.models.Request](https://requests.readthedocs.io/en/master/_modules/requests/models/#Request) - Last source version (your actual version could be lower)

In [12]:
def add_header_hook(r, *args, **kwargs):
    r.headers.update([("X-AWESOME-HEADER", "I'm Awesome Header")])
    
    return r

In [13]:
with requests.Session() as s:
    # Prepare data
    request = requests.Request("GET", "https://httpbin.org/anything")
    request = add_header_hook(request)
    prepared = request.prepare()

    # Send data
    resp = s.send(prepared)
    
resp.json()

{'args': {},
 'data': '',
 'files': {},
 'form': {},
 'headers': {'Accept-Encoding': 'identity',
  'Host': 'httpbin.org',
  'X-Amzn-Trace-Id': 'Root=1-6017bf53-723338f0284a125c64371baa',
  'X-Awesome-Header': "I'm Awesome Header"},
 'json': None,
 'method': 'GET',
 'origin': '37.214.75.32',
 'url': 'https://httpbin.org/anything'}

In [14]:
import requests
from typing import Callable


class PreRequestHooks():
    def __init__(self, *hooks, session=None):
        self._hooks = hooks
        self._session = session or requests.Session()
        self._closed = False
    
    @property
    def session(self):
        return self._session
    
    @property
    def closed(self):
        return self._closed
    
    @property
    def hooks(self):
        return self._hooks
    
    def __enter__(self):
        if self._closed:
            raise SystemError("%r object are closed" % self.__class__)
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        if not self._closed:
            self.session.close()
    
    def request(self, method, url, headers=None, files=None, 
                data=None, params=None, auth=None, cookies=None, 
                hooks=None, json=None):
        # or replace to request(method, url, *args, **kwargs)
        
        request = requests.Request(
            method, url,
            headers=headers,
            files=files,
            data=data,
            params=params,
            auth=auth,
            cookies=cookies,
            hooks=hooks,
            json=json,
        )
        
        for hook in self._hooks:
            if not isinstance(hook, Callable):
                raise TypeError("Hook expected callable object but got %r" % hook.__class__)
            request = hook(request)
            
        prepared = request.prepare()
        return self._session.send(prepared)
    
    def get(self, url, **kwargs):
        return self.request('GET', url, **kwargs)

    def options(self, url, **kwargs):
        return self.request('OPTIONS', url, **kwargs)

    def head(self, url, **kwargs):
        return self.request('HEAD', url, **kwargs)

    def post(self, url, data=None, json=None, **kwargs):
        return self.request('POST', url, data=data, json=json, **kwargs)

    def put(self, url, data=None, **kwargs):
        return self.request('PUT', url, data=data, **kwargs)

    def patch(self, url, data=None, **kwargs):
        return self.request('PATCH', url, data=data, **kwargs)

    def delete(self, url, **kwargs):
        return self.request('DELETE', url, **kwargs)


In [15]:
with requests.Session() as s:
    # With Hooks
    pre_requests = PreRequestHooks(add_header_hook, session=s)
    resp_hooks = pre_requests.get("https://httpbin.org/anything")
    print(f"{resp_hooks.json() = }")
    print()
    
    # Without hooks
    resp = s.get("https://httpbin.org/anything")
    resp.json()
    print(f"{resp.json() = }")

resp_hooks.json() = {'args': {}, 'data': '', 'files': {}, 'form': {}, 'headers': {'Accept-Encoding': 'identity', 'Host': 'httpbin.org', 'X-Amzn-Trace-Id': 'Root=1-6017c006-7c2b4b9f2799367952c2a733', 'X-Awesome-Header': "I'm Awesome Header"}, 'json': None, 'method': 'GET', 'origin': '37.214.75.32', 'url': 'https://httpbin.org/anything'}

resp.json() = {'args': {}, 'data': '', 'files': {}, 'form': {}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.24.0', 'X-Amzn-Trace-Id': 'Root=1-6017c006-600eda0b26c3a83908f570a5'}, 'json': None, 'method': 'GET', 'origin': '37.214.75.32', 'url': 'https://httpbin.org/anything'}


In [16]:
class UnsupportedMethod(ValueError):
    pass


def check_method(r, *args, **kwargs):
    if r.method.upper() not in frozenset(["GET", "HEAD"]):
        raise UnsupportedMethod("Supported 'GET' and 'HEAD' methods only but got %r" % r.method)
    return r

In [17]:
session = PreRequestHooks(
    check_method,
    add_header_hook,  
    session=requests.Session()  # as eg it coould be some session which implement Retries
)


with session:
    resp = session.get("https://httpbin.org/anything")
    
resp.json()

{'args': {},
 'data': '',
 'files': {},
 'form': {},
 'headers': {'Accept-Encoding': 'identity',
  'Host': 'httpbin.org',
  'X-Amzn-Trace-Id': 'Root=1-6017c07c-4596b7bc675f435f41ba7026',
  'X-Awesome-Header': "I'm Awesome Header"},
 'json': None,
 'method': 'GET',
 'origin': '37.214.75.32',
 'url': 'https://httpbin.org/anything'}

In [18]:
session = PreRequestHooks(
    check_method,
    add_header_hook,  
    session=requests.Session()  # as eg it coould be some session which implement Retries
)


with session:
    resp = session.post("https://httpbin.org/anything")
    
resp.json()

UnsupportedMethod: Supported 'GET' and 'HEAD' methods only but got 'POST'